In [1]:
### we start with the liar dataset

import numpy as np
import pandas as pd

In [2]:
#get the dataset
df_train = pd.read_csv('../../../datasets/liar_tweaked/trainvectordata.csv')
df_test = pd.read_csv('../../../datasets/liar_tweaked/testvectordata.csv')
df_valid = pd.read_csv('../../../datasets/liar_tweaked/validvectordata.csv')

In [3]:
X_train=df_train['statement']
X_test=df_test['statement']
Y_train=df_train['label']
Y_test=df_test['label']
X_valid=df_valid['statement']
Y_valid=df_valid['label']

In [4]:
X_test=X_test.append(X_valid, ignore_index=True)

In [5]:
Y_test=Y_test.append(Y_valid, ignore_index=True)

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical



Using TensorFlow backend.


In [7]:
X_test = X_test.str.replace('[^a-zA-Z]',' ').str.lower()
X_train = X_train.str.replace('[^a-zA-Z]',' ').str.lower()



In [8]:
import nltk
import re
from nltk.corpus import stopwords

In [9]:
# Remove stop words with regex. '\\b' matches any break (space or linebreak or whatever) and '|'
# is an or operator. So, for example '\\ba\\b|\\bis\\b|\\band\\b' will match 'a', 'is' or 'and'.
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
X_test = X_test.str.replace(stop_re, '')
X_train = X_train.str.replace(stop_re, '')
X_test.head()

0    building  wall   u   mexico border  take liter...
1     wisconsin   pace  double  number  layoffs  year 
2          says john mccain  done nothing  help  vets 
3    suzanne bonamici supports  plan   cut choice  ...
4     asked   reporter whether hes   center   crimi...
Name: statement, dtype: object

In [10]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
for i in range(0, len(X_test)):
    s=X_test[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X_test[i]=st
for i in range(0, len(X_train)):
    s=X_train[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X_train[i]=st

In [11]:
# Tokenize words
X_test = X_test.str.split()
X_train = X_train.str.split()

In [12]:
import gensim
from gensim.models import Word2Vec

In [13]:
w2vtest = gensim.models.Word2Vec(X_test.tolist(), min_count=1)
w2vtrain = gensim.models.Word2Vec(X_train.tolist(), min_count=1)

In [14]:
def transform_data(data,w2v):
    for i in range(0, len(data)):
        x=data[i]
        b = np.zeros(100) 
        for w in x:
            b+=w2v[w]
        b = np.divide(b, 100)
        x=b
        data[i]=x
        #print(b)
    return data

In [15]:
X_test=transform_data(X_test,w2vtest)
X_train=transform_data(X_train,w2vtrain)

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [16]:
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

In [17]:
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [23]:
#building the classifier
def build_classifier():
    clf=Sequential()
    clf.add(Dense(output_dim=500,init='uniform',activation='relu',input_dim=100))
    clf.add(Dense(output_dim=100,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=50,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=20,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=10,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=5,init='uniform',activation='relu'))
    clf.add(Dense(output_dim=1,init='uniform',activation='sigmoid'))
    clf.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return clf

In [24]:
#make necessary imports
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

#build ANN, use k fold cross validation
clf=KerasClassifier(build_fn=build_classifier, batch_size=10, nb_epoch=100)
accuracies=cross_val_score(estimator=clf, X=tfidf_train,y=Y_train,cv=10,n_jobs=-1)

NameError: name 'tfidf_train' is not defined

In [22]:
#fit on training data and check accuracies on both test and valid data
clf.fit(X_train,Y_train, batch_size=10, nb_epoch=10)
y_test_pred = clf.predict(X_test)
print('algorithm - test dataset accuracy - valid dataset accuracy')
print('ANNW2V - ' ,round(accuracy_score(Y_test, y_test_pred),4), ' - ', end='')

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=100, units=500, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  """
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="uniform")`
  
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20, kernel_initializer="uniform")`
  import sys
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: U

Epoch 1/10
 7810/10229 [=====================>........] - ETA: 3s - loss: 0.6874 - accuracy: 0.5622

KeyboardInterrupt: 

In [26]:
history = clf.fit(X_train, Y_train, epochs=10,
                    validation_data=(X_test, Y_test))

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=100, units=500, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  """
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="uniform")`
  
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20, kernel_initializer="uniform")`
  import sys
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: U

Train on 10229 samples, validate on 2551 samples
Epoch 1/10
10229/10229 [==============================] - 16s 2ms/step - loss: 0.6880 - accuracy: 0.5610 - val_loss: 0.6897 - val_accuracy: 0.5417
Epoch 2/10
10229/10229 [==============================] - 24s 2ms/step - loss: 0.6857 - accuracy: 0.5619 - val_loss: 0.6901 - val_accuracy: 0.5417
Epoch 3/10
10229/10229 [==============================] - 18s 2ms/step - loss: 0.6855 - accuracy: 0.5619 - val_loss: 0.6908 - val_accuracy: 0.5417
Epoch 4/10
10229/10229 [==============================] - 13s 1ms/step - loss: 0.6855 - accuracy: 0.5619 - val_loss: 0.6906 - val_accuracy: 0.5417
Epoch 5/10
10229/10229 [==============================] - 12s 1ms/step - loss: 0.6855 - accuracy: 0.5619 - val_loss: 0.6906 - val_accuracy: 0.5417
Epoch 6/10
10229/10229 [==============================] - 12s 1ms/step - loss: 0.6855 - accuracy: 0.5619 - val_loss: 0.6906 - val_accuracy: 0.5417
Epoch 7/10
10229/10229 [==============================] - 13s 1ms/ste

In [27]:
### now we have the kaggle dataset

In [28]:
df = pd.read_csv('../../../datasets/kaggle/train.csv')



In [29]:
df['content'] = df['title'] + ' ' + df['text']
df=df[df['content']==df['content']]
df=df[df['label']==df['label']]

In [30]:
df.to_csv('../../../datasets/kaggle/final.csv')

In [31]:
df=pd.read_csv('../../../datasets/kaggle/final.csv')

In [32]:
X=df['content']
Y=df['label']

In [33]:
X = X.str.replace('[^a-zA-Z]',' ').str.lower()



In [34]:
stop_re = '\\b'+'\\b|\\b'.join(nltk.corpus.stopwords.words('english'))+'\\b'
X = X.str.replace(stop_re, '')
X.head()

0    house dem aide     even see comey  letter  jas...
1    flynn  hillary clinton  big woman  campus   br...
2      truth might get  fired   truth might get  fi...
3       civilians killed  single us airstrike   ide...
4    iranian woman jailed  fictional unpublished st...
Name: content, dtype: object

In [35]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
for i in range(0, len(X)):
    s=X[i]
    st=""
    st+=" ".join([ps.stem(i) for i in s.split()])
    X[i]=st

In [36]:
# Tokenize words
X = X.str.split()

In [37]:
from sklearn.model_selection import train_test_split

In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [39]:
w2vtest = gensim.models.Word2Vec(X_test.tolist(), min_count=1)
w2vtrain = gensim.models.Word2Vec(X_train.tolist(), min_count=1)

In [40]:
X_test=X_test.reset_index()
Y_test=Y_test.reset_index()
X_train=X_train.reset_index()
Y_train=Y_train.reset_index()

In [41]:
Y_test=Y_test['label']
X_test=X_test['content']
X_train=X_train['content']
Y_train=Y_train['label']

In [42]:
X_test=transform_data(X_test,w2vtest)
X_train=transform_data(X_train,w2vtrain)

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [43]:
X_train = np.array(X_train.tolist())
X_test = np.array(X_test.tolist())

In [44]:
Y_train = np.array(Y_train.tolist())
Y_test = np.array(Y_test.tolist())

In [46]:
history = clf.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_test, Y_test))

/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=100, units=500, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=100, kernel_initializer="uniform")`
  """
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=50, kernel_initializer="uniform")`
  
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=20, kernel_initializer="uniform")`
  import sys
/Users/lovedeepsingh/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: U

Train on 15152 samples, validate on 5051 samples
Epoch 1/20
15152/15152 [==============================] - 16s 1ms/step - loss: 0.3415 - accuracy: 0.8496 - val_loss: 0.6518 - val_accuracy: 0.7597
Epoch 2/20
15152/15152 [==============================] - 13s 880us/step - loss: 0.2442 - accuracy: 0.9032 - val_loss: 1.0868 - val_accuracy: 0.7361
Epoch 3/20
15152/15152 [==============================] - 17s 1ms/step - loss: 0.2195 - accuracy: 0.9110 - val_loss: 0.5876 - val_accuracy: 0.7173
Epoch 4/20
15152/15152 [==============================] - 25s 2ms/step - loss: 0.2051 - accuracy: 0.9172 - val_loss: 0.6927 - val_accuracy: 0.7395
Epoch 5/20
15152/15152 [==============================] - 16s 1ms/step - loss: 0.1939 - accuracy: 0.9230 - val_loss: 0.6915 - val_accuracy: 0.7365
Epoch 6/20
15152/15152 [==============================] - 15s 1ms/step - loss: 0.1836 - accuracy: 0.9275 - val_loss: 0.7083 - val_accuracy: 0.7416
Epoch 7/20
15152/15152 [==============================] - 19s 1ms/s